In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Building reusable machine learning workflows with pipeline templates

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_template.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview
In this notebook you are demonstrating how you can use the Kubeflow Pipelines DSL `RegistryClient` to build, upload and reuse a ML pipeline using Vertex Pipelines and the Artifact Registry. You can always go to our [Pipeline Template documentation](https://cloud.google.com/vertex-ai/docs/pipelines/create-pipeline-template) to learn more about creating, uploading and reusing templates. 

### Dataset
We are using the [penguin dataset](https://www.tensorflow.org/datasets/catalog/penguins) that is available as a public dataset in [BigQuery](https://cloud.google.com/bigquery/public-data).

### Objective

This tutorial uses the following Google Cloud ML services and resources:
- KFP DSL (RegistryClient)
- Vertex AI SDK
- Vertex AI Pipelines
- Artifact Registry
- Vertex AI Workbench
- Vertex AI Endpoints
- BigQuery

The steps performed include:
- How to build a pipeline using the KFP DSL.
- How to use the KFP RegistryClient to upload and use a Pipeline artifact.
- Run the pipeline using Vertex AI Pipelines.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Artifact Registry
* BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Artifact Registry Pricing](https://cloud.google.com/artifact-registry/pricing), [BigQuery](https://cloud.google.com/bigquery/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install the following packages required to execute this notebook. 

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q
! pip3 install -U google-cloud-storage {USER_FLAG} -q
! pip3 install {USER_FLAG} kfp==2.0.0b1 --upgrade -q

### Check installed package versions
Run the following cell to check if you are using `kfp>=2.0.0b1`.

In [ ]:
! pip freeze | grep kfp

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs. Might take a bit.
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com) and the [Artifact Registry API](https://console.cloud.google.com/apis/library/artifactregistry.googleapis.com). **Enabling the API's will take a few minutes**

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = (
    "[your-region]"  # @param {type: "string"} --> You can change the region if you want
)

if REGION == "[your-region]":
    REGION = "us-central1"

print(f"Region: {REGION}")

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already
authenticated. 

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**
In this tutorial, Vertex AI also saves the
trained model that results from your job in the same bucket. Using this model artifact, you can then
create Vertex AI model and endpoint resources in order to serve
online predictions. We will also use the GCS bucket for our Vertex Pipeline run.  

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"} --> Set to your bucket.
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "vertex-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

print(f"Your bucket: {BUCKET_URI}")

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

### Create a repository in Artifact Registry

Next you need to create a repository in Artifact Registry. 

In [ ]:
! gcloud artifacts repositories create quickstart-kfp-repo \
    --repository-format=kfp \
    --location=us-central1 \
    --description="kfp-template-repo" 

Next verify if the repo has been created. The next cell should output a list of the repositories. The list should contain `quickstart-kfp-repo`.

In [ ]:
!gcloud artifacts repositories list --project=$PROJECT_ID \
--location=us-central1

You can also use the [Google Cloud Console](https://cloud.google.com/artifact-registry/docs/repositories/create-repos#console) to create a Artifact Repository. 

### Import libraries
Now you need to import our Python libraries. We will be using:

* [Vertex AI SDK](https://cloud.google.com/vertex-ai/docs/start/client-libraries)
* [Kubeflow Pipelines DSL](https://kubeflow-pipelines.readthedocs.io/en/latest/source/kfp.dsl.html)

In [ ]:
import google.cloud.aiplatform as vertex_ai
from kfp import compiler, dsl

In order to run our pipeline you also need to specify some constants that you can reuse in our pipeline. 

In [ ]:
TRAINING_DATA = "bigquery-public-data.ml_datasets.penguins"
PIPELINE_ROOT = BUCKET_URI + "/pipeline-root"
MODEL_DIR = BUCKET_URI + "/model-dir"
MODEL_NAME = "template_model"
PACKAGE_NAME = "pipeline-template"

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### Create three custom components 
In this example you will create a pipeline using components that are based on Python function. You can learn more about how to create a component from a Python function in the [KFP documentation](https://www.kubeflow.org/docs/components/pipelines/sdk-v2/python-function-components/). Our pipeline consists of three steps:

* Train a Tensorflow model based on data in BigQuery. 
* Upload the trained model to the Vertex AI Model Registry.   
* Create a Vertex AI Endpoint and Deploy our model into the Endpoint. 

Run the following cells that will define the component code.   

In [ ]:
@dsl.component(
    packages_to_install=[
        "pandas",
        "scikit-learn",
        "tensorflow",
        "google-cloud-bigquery-storage==2.4.0",
        "google-cloud-bigquery",
        "pyarrow",
        "fsspec",
        "gcsfs",
        "google-cloud-aiplatform==1.15.0",
        "kfp==2.0.0b1",
        "kfp-pipeline-spec",
        "kfp-server-api",
        "pandas==1.2.4",
        "pandas-profiling==3.0.0",
        "numpy==1.19.5",
    ]
)
def model_train(
    bq_data: str,
    project_id: str,
    region: str,
    model_dir: str,
):

    import logging
    import os

    import tensorflow as tf
    from google.cloud import bigquery, bigquery_storage
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder

    bqclient = bigquery.Client(project_id)
    bqstorageclient = bigquery_storage.BigQueryReadClient()

    query_string = """
    SELECT
      *
    FROM `{}`
    """.format(
        bq_data
    )

    df = (
        bqclient.query(query_string)
        .result()
        .to_dataframe(bqstorage_client=bqstorageclient)
    )

    print(df.head(2))

    output_directory = model_dir
    if os.environ.get("AIP_MODEL_DIR") is not None:
        output_directory = os.environ["AIP_MODEL_DIR"]

    print(output_directory)

    logging.info("Creating and training model ...")

    model = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(32, activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(16, activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(8, activation="relu"),
            tf.keras.layers.Dense(1, activation="softmax"),
        ]
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, clipnorm=1.0),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.AUC()],
    )

    train, test = train_test_split(df, test_size=0.2)

    print(train.head(2))

    train_X, train_y = (
        train[["island", "culmen_length_mm", "culmen_depth_mm", "body_mass_g"]],
        train["species"],
    )

    train_X[["island", "culmen_length_mm", "culmen_depth_mm", "body_mass_g"]] = train_X[
        ["island", "culmen_length_mm", "culmen_depth_mm", "body_mass_g"]
    ].fillna(0)

    enc = LabelEncoder()
    enc.fit(train_X["island"])
    train_X["island"] = enc.transform(train_X["island"])

    print(train_X.head(5))

    train_y.replace(
        [
            "Gentoo penguin (Pygoscelis papua)",
            "Chinstrap penguin (Pygoscelis antarctica)",
            "Adelie Penguin (Pygoscelis adeliae)",
        ],
        [1, 2, 3],
        inplace=True,
    )

    print(train_y.head(10))
    print(train_y.dtypes)

    model.fit(train_X, train_y, epochs=10, verbose=1, batch_size=10)

    logging.info(f"Exporting SavedModel to: {output_directory}")
    model.save(output_directory)

Next the component that will upload the model to the Vertex AI Model Registry. 

In [ ]:
@dsl.component(
    base_image="python:3.8-slim", packages_to_install=["google-cloud-aiplatform"]
)
def model_upload(
    project_id: str,
    region: str,
    model_dir: str,
    model_name: str,
):

    import google.cloud.aiplatform as vertex_ai

    vertex_ai.init(project=project_id, location=region)

    vertex_ai.Model.upload(
        display_name=model_name,
        artifact_uri=model_dir,
        serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-2:latest",
    )

And the last component will deploy our model to a Vertex AI Endpoint. 

In [ ]:
@dsl.component(
    base_image="python:3.8-slim", packages_to_install=["google-cloud-aiplatform"]
)
def model_deploy(
    project_id: str,
    region: str,
    model_name: str,
):
    import google.cloud.aiplatform as vertex_ai

    vertex_ai.init(project=project_id, location=region)

    model = vertex_ai.Model.list()
    model = model[0]

    model.deploy(
        min_replica_count=1,
        max_replica_count=1,
    )

Now you have created three components that you can reuse in our pipeline workflow. 

### Build the pipeline
The next step is that you will build our pipeline. This pipeline will be our template that you will upload and reuse later in this notebook. 

In [ ]:
@dsl.pipeline(
    name="pipeline_template",
    description="pipeline that trains and uploads a TF Model",
)
def ml_pipeline():
    train_task = model_train(
        bq_data=TRAINING_DATA, project_id=PROJECT_ID, region=REGION, model_dir=MODEL_DIR
    )

    uploader_task = model_upload(
        project_id=PROJECT_ID, region=REGION, model_dir=MODEL_DIR, model_name=MODEL_NAME
    ).after(train_task)

    _ = model_deploy(project_id=PROJECT_ID, region=REGION, model_name=MODEL_NAME).after(
        uploader_task
    )

Now you have to compile our pipeline in a local YAML file called `pipeline_template.yaml`. For this you will use the `kfp.Compiler`.

In [ ]:
compiler.Compiler().compile(
    pipeline_func=ml_pipeline, package_path="pipeline_template.yaml"
)

### Upload the template

Now it's time to use the `RegistryClient`. First you need to configure your Kubeflow Pipelines SDK registry client.

In [ ]:
from kfp.registry import RegistryClient

client = RegistryClient(
    host=f"https://us-central1-kfp.pkg.dev/{PROJECT_ID}/quickstart-kfp-repo"
)

After configuring your registry client you can go ahead and upload the `pipeline_template.yaml`.

In [ ]:
templateName, versionName = client.upload_pipeline(
    file_name="pipeline_template.yaml",
    tags=["v1", "latest"],
    extra_headers={"description": "This is an example pipeline template"},
)

!rm pipeline_template.yaml

#### Verify if the template was uploaded to the Artifact Registry repo. 
Now you can verify if the template was uploaded. 

In [ ]:
templatePackages = client.list_packages()
templatePackage = client.get_package(package_name=PACKAGE_NAME)
print(templatePackage)

You can also view the uploaded artifact in the [Artifact Registry repo](https://console.cloud.google.com/artifacts/browse/) that you created earlier. 

### Reuse your pipeline template using Vertex Pipelines

Next you will reuse the pipeline template that you just created and uploaded. You can use the Vertex AI SDK for Python or the [Google Cloud console](https://console.cloud.google.com/vertex-ai/pipelines/runs) to create a pipeline run from your template in Artifact Registry. The following command  will create submit a `PipelineJob`. 

**This will take ~15 minutes to complete**

In [ ]:
# Create a job via version id.
job = vertex_ai.PipelineJob(
    display_name="pipeline-template",
    template_path=f"https://us-central1-kfp.pkg.dev/{PROJECT_ID}/quickstart-kfp-repo/pipeline-template/"
    + versionName,
)

job.submit()

You can view the runs created by a specific pipeline version in the Vertex AI SDK for Python. To list the pipeline runs, run the `pipelineJobs.list` command as shown in one or more of the following examples:

In [ ]:
# To filter all runs created from a specific version
filter = (
    f'template_uri:"https://us-central1-kfp.pkg.dev/{PROJECT_ID}/quickstart-kfp-repo/pipeline-template/*" AND '
    + 'template_metadata.version="%s"' % versionName
)

vertex_ai.PipelineJob.list(filter=filter)

#### Check if the Endpoint has been created. 
After your Pipeline Job finishes you can check if your endpoint has been created. 
**Note**: Wait until your pipeline run is finished. This will take around ~15 minutes. 

In [ ]:
endpoint = vertex_ai.Endpoint.list(order_by="update_time")
endpoint[-1]

You can also navigate to the [Vertex Pipelines Console](https://console.cloud.google.com/vertex-ai/pipelines/) to see the submitted job run. 

You now made it to the end of this notebook!

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

* [Delete Vertex AI Model](https://cloud.google.com/vertex-ai/docs/model-registry/delete-model)
* [Delete GCS bucket](https://cloud.google.com/storage/docs/deleting-buckets)

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Delete Cloud Storage objects that were created
! gsutil -m rm -r $MODEL_DIR

if os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI